## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

# Device & Path

In [4]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
torch.backends.mps.is_available()

True

In [5]:
# PATH
DATA_PATH  = '../DATA'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH  = os.path.join(DATA_PATH, 'test')
TRAIN_CSV  = os.path.join(DATA_PATH, 'train.csv')
TEST_CSV   = os.path.join(DATA_PATH, 'test.csv')
SAMPLE_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')

## Hyperparameter Setting

In [6]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':41
}

## Fixed RandomSeed

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [22]:
import os
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
all_folder_list = os.listdir(TRAIN_PATH)
label = list(all_folder_list)
label = [x for x in label if not x.startswith('.')]
label = le.fit_transform(label)
label
# Rename folders with encoded labels
# for k, v in zip(all_folder_list, label):
#     os.rename(os.path.join(TRAIN_PATH, k), os.path.join(TRAIN_PATH, str(v)))


array([ 4,  1,  3,  9, 11, 12, 13,  7,  6, 14, 17,  2,  8,  5, 10,  0, 15,
       18, 16])

In [23]:
all_folder_list = [x for x in all_folder_list if not x.startswith('.')]
all_folder_list

['녹오염',
 '걸레받이수정',
 '꼬임',
 '석고수정',
 '오타공',
 '울음',
 '이음부불량',
 '몰딩수정',
 '면불량',
 '창틀,문틀수정',
 '피스',
 '곰팡이',
 '반점',
 '들뜸',
 '오염',
 '가구수정',
 '터짐',
 '훼손',
 '틈새과다']

In [28]:
import gc
gc.collect()

1136

In [26]:
# train folder에 있는 이미지 파일 이름을 리스트로 저장
all_folder_list = os.listdir(TRAIN_PATH)

path_list = []
all_path_list = []
for item in all_folder_list:
    if not item.startswith('.'):
        path_list = os.path.join(TRAIN_PATH, item)
        for a in os.listdir(path_list):
            if not a.startswith('.'):
                path_list_2 = os.path.join(path_list, a)
                all_path_list.append(path_list_2)

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_path_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[3])


In [27]:
df

,img_path,label
0,../DATA/train/녹오염/8.png,녹오염
1,../DATA/train/녹오염/9.png,녹오염
2,../DATA/train/녹오염/12.png,녹오염
3,../DATA/train/녹오염/13.png,녹오염
4,../DATA/train/녹오염/11.png,녹오염
...,...,...
3452,../DATA/train/틈새과다/4.png,틈새과다
3453,../DATA/train/틈새과다/2.png,틈새과다
3454,../DATA/train/틈새과다/3.png,틈새과다
3455,../DATA/train/틈새과다/1.png,틈새과다


In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [11]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
train['label'] = le.inverse_transform(train['label'])
val['label'] = le.inverse_transform(val['label'])
train

,img_path,label
1158,../DATA/train/18/236.png,18
467,../DATA/train/18/274.png,18
2506,../DATA/train/10/335.png,10
2454,../DATA/train/10/10.png,10
525,../DATA/train/18/107.png,18
...,...,...
1555,../DATA/train/11/26.png,11
619,../DATA/train/18/702.png,18
1884,../DATA/train/17/4.png,17
1355,../DATA/train/18/595.png,18


## CustomDataset

In [12]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [13]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [14]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [15]:
# !pip install --upgrade efficientnet_pytorch

In [16]:
import torch.nn as nn
import torchvision.models as models
from efficientnet_pytorch import EfficientNet

class FineTunedModel(nn.Module):
    def __init__(self, num_classes):
        super(FineTunedModel, self).__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b1')
        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        return x



# # Instantiate the model
# model = FineTunedModel(num_classes=len(set(train['label'])))

# # # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the new layers of the model
# num_epochs = 10
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # Create a dictionary to map each unique label to a numerical value
#         label_map = {label: i for i, label in enumerate(set(labels))}
#         # Convert the string labels to numerical labels using the label map
#         numerical_labels = [label_map[label] for label in labels]
#         # Create a PyTorch tensor from the numerical labels
#         labels_tensor = torch.tensor(numerical_labels, dtype=torch.long)
#         loss = criterion(outputs, labels_tensor)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * inputs.size(0)

#     # Calculate and print training loss
#     epoch_loss = running_loss / len(train_loader.dataset)
#     print('Epoch [{}/{}], Training Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

#     # Calculate validation loss
#     val_running_loss = 0.0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             outputs = model(inputs)
#             # Create a dictionary to map each unique label to a numerical value
#             label_map = {label: i for i, label in enumerate(set(labels))}
#             # Convert the string labels to numerical labels using the label map
#             numerical_labels = [label_map[label] for label in labels]
#             # Create a PyTorch tensor from the numerical labels
#             labels_tensor = torch.tensor(numerical_labels, dtype=torch.long)
#             loss = criterion(outputs, labels_tensor)
#             val_running_loss += loss.item() * inputs.size(0)

#     # Calculate and print validation loss
#     val_epoch_loss = val_running_loss / len(val_loader.dataset)
#     print('Epoch [{}/{}], Validation Loss: {:.4f}'.format(epoch+1, num_epochs, val_epoch_loss))




## Train

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            # Create a dictionary to map each unique label to a numerical value
            label_map = {label: i for i, label in enumerate(set(labels))}
            # Convert the string labels to numerical labels using the label map
            numerical_labels = [label_map[label] for label in labels]
            # Create a PyTorch tensor from the numerical labels
            labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [18]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            # Create a dictionary to map each unique label to a numerical value
            label_map = {label: i for i, label in enumerate(set(labels))}
            # Convert the string labels to numerical labels using the label map
            numerical_labels = [label_map[label] for label in labels]
            # Create a PyTorch tensor from the numerical labels
            labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [19]:
df['label']

0       18
1       18
2       18
3       18
4       18
        ..
3452    14
3453    14
3454    14
3455    14
3456    14
Name: label, Length: 3457, dtype: object

In [20]:
next(iter(train_loader))[0].shape

torch.Size([32, 3, 224, 224])

In [21]:
model = FineTunedModel(num_classes=len(set(df['label'])))
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Loaded pretrained weights for efficientnet-b1


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.38360] Val Loss : [2.28487] Val Weighted F1 Score : [0.17137]


  0%|          | 0/87 [00:00<?, ?it/s]

...

Epoch [14], Train Loss : [0.03568] Val Loss : [3.86279] Val Weighted F1 Score : [0.18844]


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.03227] Val Loss : [3.89364] Val Weighted F1 Score : [0.19458]


  0%|          | 0/87 [00:00<?, ?it/s]

## Inference

In [ ]:
test = pd.read_csv(TEST_CSV)

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv(SAMPLE_PATH)

In [ ]:
submit['label'] = preds

In [ ]:
from datetime import datetime, timedelta, timezone
# 시간 고유값 
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# 기록 경로
RECORDER_DIR = os.path.join(DATA_PATH, 'results', train_serial)

# 현재 시간 기준 폴더 생성
os.makedirs(RECORDER_DIR, exist_ok=True)    
RESULT_PATH = os.path.join(RECORDER_DIR, 'submission.csv')

In [ ]:
submit.to_csv(RESULT_PATH, index=False)